In [ ]:
%load_ext autoreload
%autoreload 2

from discontinuum.providers import usgs
from discontinuum.utils import aggregate_to_daily

import xarray as xr

In [ ]:
# setup

# Choptank River at Greensboro, MD
site = "01491000" 
start_date = "1979-10-01"
end_date = "2011-09-30"

# SF Coeur D Alene River 
site = "12413470"
start_date = "1988-10-01" 
end_date = "2021-09-30" 

characteristic = 'Inorganic nitrogen (nitrate and nitrite)'
#characteristic = 'Phosphorus'
fraction = 'Dissolved'

In [ ]:
# download data

daily = usgs.get_daily(site=site, start_date=start_date, end_date=end_date)
samples = usgs.get_samples(site=site, start_date=start_date, end_date=end_date, characteristic=characteristic, fraction=fraction)

samples = aggregate_to_daily(samples)

training_data = xr.merge([samples, daily], join='inner')

In [ ]:
%%time
# fit model

from discontinuum.models import LoadestGP

model = LoadestGP()
model.fit(target=training_data['concentration'], covariates=training_data[['time','flow']])

In [ ]:
# plot result

model.plot_concentration(daily[['time','flow']])

In [ ]:
sim = model.sample(daily[['time','flow']])

In [ ]:
flux = sim * daily['flow'] * 86400 * 1e-3 #kg/day

In [ ]:
#compute load then do
# note first year is low b/c we are pulling water years
flux.resample(time='YE').sum().to_dataframe(name='load').boxplot(by='time', showfliers=False, grid=False)